In [ ]:
import json
import numpy as np
from pycocotools import mask
import cv2
import os
import shutil
from tqdm import tqdm
import traceback

def yolo_to_abs_coords(yolo_data, img_width, img_height):
    abs_coords = []
    for i in range(0, len(yolo_data), 2):
        x = int(yolo_data[i] * img_width)
        y = int(yolo_data[i + 1] * img_height)
        abs_coords.append((x, y))
        # print(x,y)
    return abs_coords

def poly_to_bbox(polygon):
    x_coords = [p[0] for p in polygon]
    y_coords = [p[1] for p in polygon]
    xmin = min(x_coords)
    ymin = min(y_coords)
    xmax = max(x_coords)
    ymax = max(y_coords)
    return [xmin, ymin, xmax - xmin, ymax - ymin]

def encode_rle(polygon, img_height, img_width):
    global mask_img
    mask_img = np.zeros((img_height, img_width), dtype=np.uint8)
    mask_img=cv2.fillPoly(mask_img, [np.array(polygon, np.int32)], 1)
    rle = mask.encode(np.asfortranarray(mask_img))
    rle['counts'] = rle['counts'].decode('ascii')  # Convert bytes to string
    return rle

textpath='path to your yolo txt labels'
imgpath='path to your images path'
jsonpath='path to save the RLE Json labels'

c=0
id=0
for tp in tqdm(os.listdir(textpath)):
    try:
        if not tp.endswith('.txt'):
            continue
        with open(textpath+tp,'r') as t1:
            k1=t1.readlines()
        yolo_txt = k1
        try:
            ima=cv2.imread(imgpath+tp[:-4]+'.JPG')
            ima=cv2.resize(ima,(1024,1024))
            ima2=ima.copy()
        except:
            ima=cv2.imread(imgpath+tp[:-4]+'.jpg')
            ima=cv2.resize(ima,(1024,1024))
            ima2=ima.copy()
        h,w,_=ima.shape
        img_width = w
        img_height = h
        annotations = []
        
        for idx, yolo in enumerate(yolo_txt):
            data = list(map(float, yolo.split()[1:]))
            polygon = yolo_to_abs_coords(data, img_width, img_height)
            ima2=cv2.fillPoly(ima2, [np.array(polygon, np.int32)], (255,0,0))
            bbox = poly_to_bbox(polygon)
            area = bbox[2] * bbox[3]
            rle = encode_rle(polygon, img_height, img_width)
            annotation = {'id': id+1,'bbox': bbox,'area': area,'segmentation': {'counts': rle['counts'],'size': [img_height, img_width]}}
            annotations.append(annotation)
            id+=1
        result_json = {'image': {'image_id': c,'license': 1,'file_name': tp[:-4]+'.JPG','height': h,'width': w,'date_captured': '07-01-2025'},'annotations': annotations}

        with open(jsonpath+tp[:-4]+'.json', 'w') as f:
            json.dump(result_json, f, indent=None)
        cv2.imwrite(jsonpath+tp[:-4]+'.jpg',ima)
        c+=1
    except:
        print(traceback.format_exc())

In [ ]:
'''
Follow the instruction given in the below link to start your custom SAM2 Training,after creating the RLE json files
https://github.com/roboflow/notebooks/blob/main/notebooks/fine-tune-sam-2.1.ipynb

'''